In [14]:
%config IPCompleter.greedy=True

### Tensors

In [74]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x=torch.tensor([[1,1],[-1,-1]])
print(x)
y=torch.tensor([[0,0],[1,1]])
print(y)
print(torch.add(x,y))
print(y.add_(x))
x0 = x.view(4)
print(x0)
x=torch.empty(5,3)
z = torch.randn(4,4,4)
z0 = z.view(-1,8,2)
print(z0.size())
print(torch.cuda.is_available())

tensor([[ 1,  1],
        [-1, -1]])
tensor([[0, 0],
        [1, 1]])
tensor([[1, 1],
        [0, 0]])
tensor([[1, 1],
        [0, 0]])
tensor([ 1,  1, -1, -1])
torch.Size([4, 8, 2])
False


###  AutoGrad

In [42]:
x = torch.rand(2,2,requires_grad=True)
#print(x)
y = x + 2
print(y.grad_fn)
z = y * y * 3
z_mean = z.mean()
print(z)
print(z_mean)
z_mean.backward()
print(x.grad)

tensor([[14.7668, 20.4317],
        [17.1685, 23.1552]], grad_fn=<MulBackward0>)
tensor(18.8806, grad_fn=<MeanBackward1>)
tensor([[3.3279, 3.9146],
        [3.5884, 4.1673]])


In [43]:
x = torch.randn(3,3,requires_grad=True)
print(x)

tensor([[-0.7367, -1.2929,  0.1704],
        [-0.4890, -0.2523,  0.0469],
        [-0.2459,  0.2969, -2.6363]], requires_grad=True)


### Neural Networks

In [114]:
from torch.autograd import Variable
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(28,15)
        self.fc2 = nn.Linear(15,10)
        self.fc3 = nn.Linear(10,10)
        
    def forward(self,x):
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        return x
    
    
    def train(self, x_train, y_train, epochs=13, bs=64, criterion=nn.MSELoss()):
        opt = optim.SGD(self.parameters(), lr=0.01)
        for epoch in range(epochs):
            print(epoch)
            n = 0
            var = Variable(x_train)
            list1 = list(var.size)
            for i in range(0,size(),bs):
                print("test")
                batch = x_train[i:i+bs]
                ybatch = y_train[i:i+bs]
                pred = model(batch)
                loss = criterion(pred, y_train)
                print("loss=", loss)
                ###
                loss.backward()
                opt.step()
                opt.zero_grad() 
                
            
            
            


### MNIST TASK

In [87]:
from pathlib import Path
import requests

# Extracting the data
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
# Creating tensors instead of numpy arrays
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid))
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


In [115]:
net=Net()
net.train(x_train,y_train)

0


TypeError: 'builtin_function_or_method' object is not iterable